## Расчёт позиций маркеров в системе координат полигона

На поле расположено 7 маркеров, координаты которых в СК полигона известны. Три маркера расположены в углах поля, по ним должна производиться "калибровка" камеры. Еще 4 маркера лежат в произвольных местах, но их координаты надо вычислить через найденную во время "калибровки" матрицу трансформации, после чего сравнить с реальными координатами. Задача в том, чтобы добиться минимальной погрешности в определении координат.

В файле `data.csv` содержится 7 столбцов со следующей структурой:

- **id**:int - id aruco-маркера в словаре ARUCO_DICT_5X5_250
- **rvec**:nparray - R-вектор координат маркера
- **tvec**:nparray - T-вектор координат маркера. Причем $T = \begin{bmatrix} x & y & z\end{bmatrix}$, где $x,y,z$ - это координаты маркера в СК камеры
- **R**:nparray - матрица поворота $R_{3x3} $ (точнее 9 её элементов)
- **roll** - 
- **pitch** - 
- **yaw** - 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

Начальные данные таковы:
![img](img1.png)

In [21]:
#aruco 72
m72_real_x = 30
m72_real_y = 30
m72_real_z = 0

# aruco 74
m74_real_x = 370
m74_real_y = 30
m74_real_z = 0

#aruco 71
m71_real_x = 30
m71_real_y = 530
m71_real_z = 0

#aruco 73 dont used in this experiment
m73_real_x = 370
m73_real_y = 530
m73_real_z = 0

#aruco 171
m171_real_x = 100
m171_real_y = 120
m171_real_z = 0

#aruco 172
m172_real_x = 270
m172_real_y = 380
m172_real_z = 0

#aruco 173
m173_real_x = 220
m173_real_y = 190
m173_real_z = 0

#aruco 174
m174_real_x = 100
m174_real_y = 400
m174_real_z = 0

In [22]:
class Marker():
    def __init__(self, marker_id, data, real_pos):
        """
        real_pos - tuple with real x, y, z coordinates
        """
        mydata = data[data['id']==marker_id]
        self.marker_id = marker_id
        self.rvecs = [list(map(lambda x: float(x), re.split("\s+", i.strip(" []")))) for i in mydata['rvec']]
        self.tvecs = [list(map(lambda x: float(x), re.split("\s+", i.strip(" []")))) for i in mydata['tvec']]
        self.Rs = [list(map(lambda x: float(x), re.split("\s+", i.strip(" []")))) for i in mydata['R']]
        self.rolls = [float(e) for e in mydata['roll']]
        self.pitches = [float(e) for e in mydata['pitch']]
        self.yaws = [float(e) for e in mydata['yaw']]
        
        #look at means with numpy
        self.rvec = np.mean(np.array(self.rvecs, dtype=np.float32), 0)
        self.tvec = np.mean(np.array(self.rvecs, dtype=np.float32), 0)
        self.R = np.mean(np.array(self.Rs, dtype=np.float32), 0).reshape(3, 3)
        self.roll = np.mean(np.array(self.rolls, dtype=np.float32), 0)
        self.pitch = np.mean(np.array(self.pitches, dtype=np.float32), 0)
        self.yaw = np.mean(np.array(self.yaws, dtype=np.float32), 0)
        
        self.real_position = np.array(real_pos)
        

In [23]:
data = pd.read_csv("data.csv", delimiter=";")

Определим каждый маркер, как отдельный объект, так будет проще читать

In [25]:
m71 = Marker(71, data, (m71_real_x, m71_real_y, m71_real_z))
m72 = Marker(72, data, (m72_real_x, m72_real_y, m72_real_z))
#m73 = Marker(73, data, (m73_real_x, m73_real_y, m73_real_z))
m74 = Marker(74, data, (m74_real_x, m74_real_y, m74_real_z))

m171 = Marker(171, data, (m171_real_x, m171_real_y, m171_real_z))
m172 = Marker(172, data, (m172_real_x, m172_real_y, m172_real_z))
m173 = Marker(173, data, (m173_real_x, m173_real_y, m173_real_z))
m174 = Marker(174, data, (m174_real_x, m174_real_y, m174_real_z))

In [58]:
half_size = 6/2 # [mm]
Rt = m173.R.transpose()

x72_axis = np.concatenate([(Rt[0] * half_size) + m72.tvec, [0]])
y72_axis = np.concatenate([(Rt[1] * half_size) + m72.tvec, [0]])
z72_axis = np.concatenate([(Rt[2] * 0) + m72.tvec, [0]])
t72_axis = np.concatenate([m72.tvec, [1]])

M = np.concatenate([x72_axis, y72_axis, z72_axis, t72_axis]).reshape(4,4)

t173_axis = np.concatenate([m172.tvec, [1]])
np.matmul(t173_axis, np.linalg.inv(M))

array([-0.0275262 , -0.03193714,  0.11077833,  1.        ])

In [76]:
half_size = 6/2 # [mm]
R = m72.R
tvec = m72.tvec

x72_axis = np.concatenate([R[0]*half_size + tvec, [0]])
y72_axis = np.concatenate([R[1]*half_size + tvec, [0]])
z72_axis = np.concatenate([R[2]*0 + tvec, [0]])
t72_axis = np.concatenate([tvec, [1]])

M = np.concatenate([x72_axis, y72_axis, z72_axis, t72_axis]).reshape(4,4)
print(M)

t_axis = np.concatenate([m74.tvec, [1]])
print(np.matmul(t_axis, np.linalg.inv(M)))
print(m74.real_position)

[[ 1.03681648  4.87848234 -0.40592548  0.        ]
 [ 3.64572644  2.51501656 -2.77367592  0.        ]
 [ 1.56038785  1.93882668 -0.69546354  0.        ]
 [ 1.56038785  1.93882668 -0.69546354  1.        ]]
[-0.00982803 -0.00623688  0.03080761  1.        ]
[370  30   0]


In [ ]:
[-0.17185973,
  0.98206893,
  0.07749094,
  0.70365828,
  0.17742803,
  -0.6880293,
  -0.68944127,
  -0.06371739,
  -0.72153367]